# Imports

In [1]:
library(edgeR)
library(biomaRt)

Loading required package: limma


# Functions

In [2]:
make_all_de = function(fit, contr.matrix, silent=FALSE, write_csv=FALSE, out=''){

res = data.frame()
res2 = list()

for (i in colnames(contr.matrix)){
    lrt <- glmLRT(fit, contrast=contr.matrix[,i])
    if (!silent){
    message(i)
    print(summary(de <- decideTestsDGE(lrt, p=0.05, lfc=1)))
    }
    lrt.tops = topTags(lrt, p=0.05, n=Inf)
    
    if (length(lrt.tops) != 0){
    lrt.tops$table = lrt.tops$table[abs(lrt.tops$table$logFC) >= 1,]
    names(lrt.tops$table) = paste(names(lrt.tops$table), 'LRT', i, sep='_')
    res2[[i]] = lrt.tops$table
    res = merge(res, lrt.tops$table, by=0, all=TRUE)
    rownames(res)=res[,1]; res = res[,-1]
    }
    
}
if (write_csv){
    res = annotate(res)
    write.csv(res, 'table_s1_from_R.csv')
    }
message("Total number of DE genes")
print(dim(res))

if (out=='table'){
    return(res)
}
else{
    return(res2)
}
    
}

annotate = function(res){
    mart <- useMart(biomart = "ensembl", dataset = "tguttata_gene_ensembl")
    annot <- getBM(attributes = c("ensembl_gene_id", "external_gene_name","description","chromosome_name","start_position", "end_position", "strand"), filter="ensembl_gene_id", values=rownames(res),mart=mart)

    rownames(annot)=annot$ensembl_gene_id
    annot = annot[,-1]

    res = cbind(res, annot[rownames(res),])
    return(res)
}

# Input

In [3]:
countFile = "input/featurecounts_tg89_clean"
groupsFile ="input/samples_groups.txt"
groupChoice = 4

counts <- read.delim(countFile, row.names=1, check.names=F)
group <- factor(read.delim( groupsFile, header=F )[,groupChoice])

# DR

In [4]:
y = DGEList(counts=counts, group=group)

keep <- rowSums(cpm(y) > 1) >= 3
table(keep)

y = y[keep, ,keep.lib.sizes=FALSE]


design = model.matrix(~0+group)
colnames(design) <- gsub("group", "", colnames(design))

keep
FALSE  TRUE 
 5721 12897 

In [5]:
y <- calcNormFactors(y)

y <- estimateGLMCommonDisp(y,design)
y <- estimateGLMTrendedDisp(y,design)
y <- estimateGLMTagwiseDisp(y,design)

fit <- glmFit(y, design)

## Comparisons

### Show NO differences between ORJU_V and SCJU_V

In [6]:
contr.matrix <- makeContrasts(ORJU_V_VS_SLJU_V = ORJU_V-SCJU_V, levels=design)

In [7]:
lrt <- glmLRT(fit, contrast=contr.matrix[,1])
print(summary(de <- decideTestsDGE(lrt, p=0.05)))

       [,1] 
Down       0
NotSig 12897
Up         0


**CCL** NO genes detected DR between ORJU_V and SCJU_V at FDR < 0.05 and NO lFC thresholds

### Comparison within subspecies between tracts

In [8]:
contr.matrix <- makeContrasts(
ORJU_H_VS_ORJU_B = ORJU_H-ORJU_B,
ORJU_H_VS_ORJU_F = ORJU_H-ORJU_F,
ORJU_H_VS_ORJU_V = ORJU_H-ORJU_V,
ORJU_B_VS_ORJU_F = ORJU_B-ORJU_F,
ORJU_F_VS_ORJU_V = ORJU_F-ORJU_V,
ORJU_B_VS_ORJU_V = ORJU_B-ORJU_V,
SCJU_H_VS_SCJU_B = SCJU_H-SCJU_B,
SCJU_H_VS_SCJU_F = SCJU_H-SCJU_F,
SCJU_H_VS_SCJU_V = SCJU_H-SCJU_V,
SCJU_B_VS_SCJU_F = SCJU_B-SCJU_F,
SCJU_B_VS_SCJU_V = SCJU_B-SCJU_V,
SCJU_F_VS_SCJU_V = SCJU_F-SCJU_V,
ORJU_B_VS_SCJU_B = ORJU_B-SCJU_B,
ORJU_H_VS_SCJU_H = ORJU_H-SCJU_H,
ORJU_F_VS_SCJU_F = ORJU_F-SCJU_F,
ORJU_V_VS_SCJU_V = ORJU_V-SCJU_V,
levels=design)

In [9]:
res = make_all_de(fit, contr.matrix, write_csv=TRUE)

ORJU_H_VS_ORJU_B


       [,1] 
Down      49
NotSig 12831
Up        17


ORJU_H_VS_ORJU_F


       [,1] 
Down      65
NotSig 12801
Up        31


ORJU_H_VS_ORJU_V


...

       [,1] 
Down       1
NotSig 12893
Up         3


ORJU_V_VS_SCJU_V


       [,1] 
Down       0
NotSig 12897
Up         0


Total number of DE genes


[1] 349  76


In [10]:
contr.matrix <- makeContrasts(
ORJU_H_VS_ORJU_B = ORJU_H-ORJU_B,
ORJU_H_VS_ORJU_F = ORJU_H-ORJU_F,
ORJU_H_VS_ORJU_V = ORJU_H-ORJU_V,
ORJU_B_VS_ORJU_F = ORJU_B-ORJU_F,
ORJU_F_VS_ORJU_V = ORJU_F-ORJU_V,
ORJU_B_VS_ORJU_V = ORJU_B-ORJU_V,
    levels=design)

res_orju = make_all_de(fit, contr.matrix, silent=TRUE, out='table')

contr.matrix <- makeContrasts(
SCJU_H_VS_SCJU_B = SCJU_H-SCJU_B,
SCJU_H_VS_SCJU_F = SCJU_H-SCJU_F,
SCJU_H_VS_SCJU_V = SCJU_H-SCJU_V,
SCJU_B_VS_SCJU_F = SCJU_B-SCJU_F,
SCJU_B_VS_SCJU_V = SCJU_B-SCJU_V,
SCJU_F_VS_SCJU_V = SCJU_F-SCJU_V,
levels=design)

res_scju = make_all_de(fit, contr.matrix, silent=TRUE, out='table')

Total number of DE genes


[1] 304  30


Total number of DE genes


[1] 112  25


Description of intersection /  differences:

In [11]:
table(row.names(res_orju) %in% row.names(res_scju))
table(row.names(res_scju) %in% row.names(res_orju))


FALSE  TRUE 
  234    70 


FALSE  TRUE 
   42    70 

## Export Gene lists for GO

In [13]:
go_outdir = 'genesIDs_for_GO/'
dir.create(go_outdir)

for (n in names(res)){
    outpath = paste0(go_outdir, n)
    write(file=outpath, rownames(res[[n]]))
}